<a href="https://colab.research.google.com/github/Pritam-Mondal18/Machine-Learning-yt/blob/main/titanic_using_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder,MinMaxScaler
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier

In [ ]:
df=pd.read_csv('train.csv')

In [ ]:
df.sample(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
462,463,0,1,"Gee, Mr. Arthur H",male,47.0,0,0,111320,38.50,E63,S
297,298,0,1,"Allison, Miss. Helen Loraine",female,2.0,1,2,113781,151.55,C22 C26,S


In [ ]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [ ]:
df.drop(['PassengerId','Name','Ticket','Cabin'],inplace=True,axis=1)

In [ ]:
df.sample(2)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
879,1,1,female,56.0,0,1,83.1583,C
404,0,3,female,20.0,0,0,8.6625,S


In [ ]:
# Step-1 train_test_split
X = df.drop(columns=['Survived'])
y = df['Survived']
X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=42)

In [ ]:
X_train

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S
...,...,...,...,...,...,...,...
106,3,female,21.0,0,0,7.6500,S
270,1,male,NaN,0,0,31.0000,S
860,3,male,41.0,2,0,14.1083,S
435,1,female,14.0,1,2,120.0000,S


In [ ]:
y_train.head(1)

,Survived
331,0


In [ ]:
# imputation transform
trf1 = ColumnTransformer([
    ('impute_age',SimpleImputer(),[2]),
    ('imputer_embarked',SimpleImputer(strategy='most_frequent'),[6])
],remainder='passthrough')

In [ ]:
# one hot encoding
trf2 = ColumnTransformer([
    ('ohe_sex_embarked',OneHotEncoder(sparse_output=False,handle_unknown='ignore'),[1,3])
],remainder='passthrough')

In [ ]:
#Scaling
trf3 = ColumnTransformer([
    ('scale',MinMaxScaler(),slice(0,10))
],remainder='passthrough')

In [ ]:
# feture selection
trf4 = SelectKBest(score_func=chi2,k=8)

In [ ]:
#train the model
trf5 = DecisionTreeClassifier()

# Create Pipeline

In [ ]:
pipe = Pipeline([
    ('trf1',trf1),
     ('trf2',trf2),
      ('trf3',trf3),
       ('trf4',trf4),
        ('trf5',trf5)
    ])

#PipeLine vs make_pipeline
pipeline requires naming of steps,make_pipeline does not.

(Same applied to ColumnTransformer vs make_column_transformer )

In [ ]:
#Alternative syntax
# pipe = make_pipeline(trf1,trf2,trf3,trf4,trf5)

In [ ]:
# train
pipe.fit(X_train,y_train)

Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('imputer_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('columntransformer-2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 3])])),
                ('columntransformer-3',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('selectkbest',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x7ef697a68720>)),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

# Explore the pipeline

In [ ]:
pipe.named_steps['columntransformer-1'].transformers_[1][1].statistics_

array(['S'], dtype=object)

In [ ]:
# Display the pipe line
from sklearn import set_config
set_config(display='diagram')

In [ ]:
# predict
y_pred = pipe.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.7877094972067039

# Cross Validation using Pipeline


In [ ]:
# cross validation using cross_val_score
from sklearn.model_selection import cross_val_score
cross_val_score(pipe, X_train, y_train, cv=5, scoring='accuracy').mean()

np.float64(0.786673889490791)

GridSearch using pipeline

In [ ]:
# GridSearchcv.
params = {
    'trf5__max_depth':[1,2,3,4,5,None]
}

In [ ]:
from sklearn.model_selection import GridSearchCV

# Correct the parameter name in the params dictionary
# The DecisionTreeClassifier step is internally named 'decisiontreeclassifier'
params = {
    'decisiontreeclassifier__max_depth':[1,2,3,4,5,None]
}

grid = GridSearchCV(pipe, params, cv=5, scoring='accuracy')
grid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('columntransformer-1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('impute_age',
                                                                         SimpleImputer(),
                                                                         [2]),
                                                                        ('imputer_embarked',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [6])])),
                                       ('columntransformer-2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe_sex_embarked',
                                                                         OneHotEncoder(handle_unkn...
                                                                                       sparse_output=False),
                                                                         [1,
                                                                          3])])),
                                       ('columntransformer-3',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('scale',
                                                                         MinMaxScaler(),
                                                                         slice(0, 10, None))])),
                                       ('selectkbest',
                                        SelectKBest(k=8,
                                                    score_func=<function chi2 at 0x7ef697a68720>)),
                                       ('decisiontreeclassifier',
                                        DecisionTreeClassifier())]),
             param_grid={'decisiontreeclassifier__max_depth': [1, 2, 3, 4, 5,
                                                               None]},
             scoring='accuracy')

In [ ]:
grid.best_score_

np.float64(0.8033093666896484)

In [ ]:
grid.best_params_

{'decisiontreeclassifier__max_depth': 3}

#Exporting pipeline


In [ ]:
#export
import pickle
pickle.dump(pipe,open('pipe.pkl','wb'))

# For production purpose

In [ ]:
pipe = pickle.load(open('pipe.pkl','rb'))

In [ ]:
# assume user input
test_input2 = np.array([2, 'male', 31.0, 0, 0, 10.5, 'S'],dtype=object).reshape(1,7)

In [ ]:
pipe.predict(test_input2)

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(


array([0])